# SHMR Datasets: Data Provenance Example

This notebook demonstrates how to document the provenance of SHMR datasets, including both downloaded observational data and calculated theoretical relations.

## Overview

This example shows how to:
1. Create SHMR datasets with complete metadata
2. Document data sources and methods
3. Save datasets in standard formats
4. Validate and visualize the data

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add the src directory to Python path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from shmr_datasets import (
    SHMRData, SHMRMetadata, 
    save_shmr, load_shmr, validate_shmr,
    calculate_shmr, interpolate_shmr
)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Example 1: Creating a Dataset from Observational Data

This example shows how to create a dataset from observational measurements with complete provenance documentation.

In [ ]:
# Simulate some observational data points
# In reality, these would come from parsing published data tables

# Example data from a hypothetical abundance matching study
log_mh_obs = np.array([10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5])
log_ms_obs = np.array([7.2, 8.5, 9.5, 10.2, 10.6, 10.8, 10.9, 11.0, 11.1])
log_ms_err = np.array([0.3, 0.2, 0.15, 0.12, 0.15, 0.18, 0.22, 0.25, 0.3])

# Convert to linear units
halo_masses_obs = 10**log_mh_obs
stellar_masses_obs = 10**log_ms_obs
stellar_mass_errors = stellar_masses_obs * log_ms_err * np.log(10)  # Convert log errors

print(f"Observational data: {len(halo_masses_obs)} data points")
print(f"Halo mass range: {halo_masses_obs.min():.1e} - {halo_masses_obs.max():.1e} Msun")
print(f"Stellar mass range: {stellar_masses_obs.min():.1e} - {stellar_masses_obs.max():.1e} Msun")

In [ ]:
# Create comprehensive metadata for the observational dataset

obs_metadata = SHMRMetadata(
    name="Example Abundance Matching SHMR",
    version="1.0",
    description="Stellar mass - halo mass relation from abundance matching analysis of SDSS galaxies",
    source_type="observation",
    reference="Smith, J. et al. 2024, 'Abundance matching in the local universe', ApJ, 999, 123",
    doi="10.3847/1538-4357/example",
    url="https://example.com/data/smith2024_shmr.dat",
    creation_method="extraction",
    creation_date=datetime.now().strftime("%Y-%m-%d"),
    created_by="Notebook Example",
    redshift=0.05,  # Mean redshift of the sample
    stellar_mass_definition="Total stellar mass within Petrosian radius",
    halo_mass_definition="M200c from weak lensing + galaxy clustering",
    cosmology={
        "h": 0.7,
        "omega_m": 0.3,
        "omega_lambda": 0.7,
        "sigma_8": 0.8
    },
    mass_range={
        "halo_mass_min": halo_masses_obs.min(),
        "halo_mass_max": halo_masses_obs.max(),
        "stellar_mass_min": stellar_masses_obs.min(),
        "stellar_mass_max": stellar_masses_obs.max()
    },
    uncertainties_included=True,
    systematic_errors="Includes uncertainties from stellar mass estimates, halo mass modeling, and sample variance",
    limitations="Limited to central galaxies with M* > 10^7 Msun; systematic uncertainties in stellar masses at low masses",
    tags=["abundance_matching", "central_galaxies", "sdss", "z0"],
    notes="Data extracted from Table 2 of Smith+ 2024. Error bars represent 1-sigma uncertainties combining statistical and systematic errors."
)

print("Created metadata for observational dataset:")
print(f"  Name: {obs_metadata.name}")
print(f"  Reference: {obs_metadata.reference}")
print(f"  Source type: {obs_metadata.source_type}")
print(f"  Creation method: {obs_metadata.creation_method}")

In [ ]:
# Create the SHMR data object

obs_shmr = SHMRData(
    halo_mass=halo_masses_obs,
    stellar_mass=stellar_masses_obs,
    stellar_mass_err_lower=stellar_mass_errors,
    stellar_mass_err_upper=stellar_mass_errors,
    metadata=obs_metadata
)

print(f"Created SHMR dataset with {obs_shmr.n_points} points")
print(f"Has stellar mass errors: {obs_shmr.has_stellar_mass_errors()}")
print(f"Has halo mass errors: {obs_shmr.has_halo_mass_errors()}")

## Example 2: Creating a Dataset from Theoretical Calculations

This example shows how to create a dataset from a parametric SHMR model.

In [ ]:
# Calculate Behroozi et al. 2013 SHMR

# Define halo mass range for calculation
log_mh_theory = np.linspace(10.0, 15.0, 100)
mh_theory = 10**log_mh_theory

# Behroozi+ 2013 parameters
behroozi_params = {
    "log_m1": 12.35,
    "ms0": 10.72,
    "beta": 0.44,
    "delta": 0.57,
    "gamma": 1.56
}

# Calculate the SHMR
theory_shmr = calculate_shmr(
    halo_masses=mh_theory,
    shmr_function="behroozi2013",
    parameters=behroozi_params,
    name="Behroozi et al. 2013 SHMR",
    version="1.0",
    description="Theoretical SHMR from Behroozi+ 2013 abundance matching parametrization",
    reference="Behroozi, P. S., Wechsler, R. H., & Conroy, C. 2013, ApJ, 770, 57",
    doi="10.1088/0004-637X/770/1/57",
    creation_date=datetime.now().strftime("%Y-%m-%d"),
    created_by="Notebook Example",
    redshift=0.0,
    stellar_mass_definition="Total stellar mass",
    halo_mass_definition="M200c",
    cosmology={"h": 0.7, "omega_m": 0.27, "omega_lambda": 0.73},
    tags=["abundance_matching", "behroozi2013", "theory"]
)

print(f"Calculated theoretical SHMR with {theory_shmr.n_points} points")
print(f"Parameter values used: {behroozi_params}")

## Data Validation

Always validate datasets to ensure they conform to the standard format.

In [ ]:
# Validate both datasets

print("Validating observational dataset:")
obs_validation = validate_shmr(obs_shmr)
print(f"  Valid: {obs_validation['valid']}")
if obs_validation['errors']:
    print(f"  Errors: {obs_validation['errors']}")
if obs_validation['warnings']:
    print(f"  Warnings: {obs_validation['warnings']}")

print("\nValidating theoretical dataset:")
theory_validation = validate_shmr(theory_shmr)
print(f"  Valid: {theory_validation['valid']}")
if theory_validation['errors']:
    print(f"  Errors: {theory_validation['errors']}")
if theory_validation['warnings']:
    print(f"  Warnings: {theory_validation['warnings']}")

## Data Visualization

Visualize the datasets to check for physical reasonableness.

In [ ]:
# Create comparison plot

fig, ax = plt.subplots(figsize=(12, 8))

# Plot observational data with error bars
ax.errorbar(
    obs_shmr.halo_mass, obs_shmr.stellar_mass,
    yerr=[obs_shmr.stellar_mass_err_lower, obs_shmr.stellar_mass_err_upper],
    fmt='o', color='red', markersize=6, capsize=3, capthick=1.5,
    label=f'{obs_shmr.metadata.name} (observations)'
)

# Plot theoretical curve
ax.plot(
    theory_shmr.halo_mass, theory_shmr.stellar_mass,
    'b-', linewidth=2.5, alpha=0.8,
    label=f'{theory_shmr.metadata.name} (theory)'
)

# Formatting
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Halo Mass [M☉]')
ax.set_ylabel('Stellar Mass [M☉]')
ax.set_title('Stellar Mass - Halo Mass Relations')
ax.legend()
ax.grid(True, alpha=0.3)

# Add text box with dataset information
info_text = (
    f"Observational data: {obs_shmr.n_points} points\n"
    f"Theoretical curve: {theory_shmr.n_points} points\n"
    f"Redshift: z = {obs_shmr.metadata.redshift}"
)
ax.text(0.05, 0.95, info_text, transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

print("\nDataset comparison plotted successfully!")

## Saving Datasets

Save datasets in multiple formats for different use cases.

In [ ]:
# Create output directory
output_dir = Path("../data/examples")
output_dir.mkdir(parents=True, exist_ok=True)

# Save observational dataset
obs_base = "example_observations_z005"
save_shmr(obs_shmr, output_dir / f"{obs_base}.h5", format="hdf5")
save_shmr(obs_shmr, output_dir / f"{obs_base}.yaml", format="yaml")

print(f"Saved observational dataset:")
print(f"  HDF5: {output_dir / obs_base}.h5")
print(f"  YAML: {output_dir / obs_base}.yaml")

# Save theoretical dataset  
theory_base = "behroozi2013_z0"
save_shmr(theory_shmr, output_dir / f"{theory_base}.h5", format="hdf5")
save_shmr(theory_shmr, output_dir / f"{theory_base}.yaml", format="yaml")

print(f"\nSaved theoretical dataset:")
print(f"  HDF5: {output_dir / theory_base}.h5")
print(f"  YAML: {output_dir / theory_base}.yaml")

## Loading and Interpolation Example

Demonstrate how to load datasets and perform interpolation.

In [ ]:
# Load a dataset back from file
loaded_shmr = load_shmr(output_dir / f"{theory_base}.h5")

print(f"Loaded dataset: {loaded_shmr.metadata.name}")
print(f"Points: {loaded_shmr.n_points}")
print(f"Reference: {loaded_shmr.metadata.reference}")

# Interpolate to new halo masses
new_halo_masses = np.logspace(11.5, 13.5, 20)
interpolated_shmr = interpolate_shmr(
    loaded_shmr, 
    new_halo_masses, 
    method="log-linear"
)

print(f"\nInterpolated to {interpolated_shmr.n_points} new points")
print(f"New halo mass range: {new_halo_masses.min():.1e} - {new_halo_masses.max():.1e}")

In [ ]:
# Plot original vs interpolated
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(loaded_shmr.halo_mass, loaded_shmr.stellar_mass, 
        'b-', linewidth=2, label='Original')
ax.plot(interpolated_shmr.halo_mass, interpolated_shmr.stellar_mass, 
        'ro', markersize=4, label='Interpolated points')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Halo Mass [M☉]')
ax.set_ylabel('Stellar Mass [M☉]')
ax.set_title('Interpolation Example')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **Complete Data Provenance**: How to document the origin, processing, and limitations of SHMR datasets
2. **Standardized Format**: Using the SHMR data format with comprehensive metadata
3. **Validation**: Ensuring data quality and format compliance
4. **Multiple Formats**: Saving data in HDF5, YAML, and JSON formats
5. **Data Manipulation**: Loading, interpolating, and visualizing SHMR datasets

This approach ensures that all SHMR datasets in the repository have complete traceability and can be easily used for scientific analyses or integration with simulation codes like Galacticus.